In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.utils.data import DataLoader, TensorDataset

np.random.seed(1337)

# Load and process the data
aggregate_df1 = pd.read_csv('/home/awinlab/Documents/kevin/elec110.csv', index_col='datetime', parse_dates=True)
aggregate_df1 = aggregate_df1[:900000]

plug1_1 = pd.read_csv('/home/awinlab/Documents/kevin/plug1-1.csv', index_col='datetime', parse_dates=True)
plug1_1 = plug1_1[:900000]

aggregate_df1 = aggregate_df1.drop(['id','device', 'vo', 'cu', 'reactive', 'apparent', 'pf', 'freq'], axis=1)
plug1_1 = plug1_1.drop(['id','device', 'vo', 'cu', 'reactive', 'apparent', 'pf', 'freq'], axis=1)

In [6]:
import torch
torch.__version__

'1.13.1+cu117'

In [2]:
print(torch.version.cuda)

11.7


In [4]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, length=100):
        self.X = X
        self.y = y
        self.length = length

    def __len__(self):
        return len(self.X) - self.length

    def __getitem__(self, idx):
        X = self.X[idx: idx + self.length]
        y = self.y[idx: idx + self.length]
        return X, y

mask = [i % 5 != 0 for i in range(len(aggregate_df1))]
X_train = aggregate_df1[~np.array(mask)]
X_test = aggregate_df1[mask]



mask1 = [i % 5 != 0 for i in range(len(plug1_1))]
y_train = plug1_1[~np.array(mask1)]
y_test = plug1_1[mask1]


X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)


X_train = torch.tensor(X_train.values).float()
X_test = torch.tensor(X_test.values).float()
y_train = torch.tensor(y_train.values).float()
y_test = torch.tensor(y_test.values).float()

batch_size=60

train_dataset = TimeSeriesDataset(X_train, y_train, length=100)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers = 0, drop_last = True, 
                                           collate_fn=lambda x: (torch.nn.utils.rnn.pad_sequence([i[0] for i in x], batch_first=True),
                                                                 torch.nn.utils.rnn.pad_sequence([i[1] for i in x], batch_first=True)))

test_dataset = TimeSeriesDataset(X_test, y_test, length=100)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 0, drop_last = True, 
                                           collate_fn=lambda x: (torch.nn.utils.rnn.pad_sequence([i[0] for i in x], batch_first=True),
                                                                 torch.nn.utils.rnn.pad_sequence([i[1] for i in x], batch_first=True)))

In [5]:
def rete(y_true, y_pred):
    y_true = y_true.view(-1, 1)
    y_pred = y_pred.view(-1, 1)

    # Compute the maximum value between y_true and y_pred element-wise, and reshape it to have shape (batch_size*seq_len, 1)
    max_val, _ = torch.max(torch.cat([y_pred, y_true], dim=1), dim=1, keepdim=True)

    # Compute the relative error between y_true and y_pred element-wise, and reshape it to have shape (batch_size*seq_len, 1)
    rel_err = torch.abs(y_pred - y_true) / max_val

    # Take the mean of the relative error across all samples in the batch
    return torch.mean(rel_err)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self, input_shape, n_input):
        super(Model, self).__init__()
        self.conv0 = nn.Conv1d(100, 16, kernel_size=1, stride=1, padding=1)
        self.conv1 = nn.Conv1d(16, 4, kernel_size=1, stride=1, padding=1)
        self.gru0 = nn.GRU(5, 64, num_layers=2, bidirectional=True)
        self.gru1 = nn.GRU(128, 128, num_layers=2, bidirectional=True)
        self.gru2 = nn.GRU(256, 256, num_layers=2, bidirectional=True)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.relu(x)
        
        x, _ = self.gru0(x)

        x, _ = self.gru1(x)
        x = self.relu(x)
        
        x, _ = self.gru2(x)
        x = self.relu(x)
        x = self.fc1(x[:, -1, :])
        x = self.fc2(x)
        return x


n_input = 100
input_shape = (n_input, 1)
model = Model(input_shape, n_input)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08)
mae_fn = nn.L1Loss()

import torch.nn.functional as F
from torch.nn import L1Loss
import matplotlib.pyplot as plt
import tqdm


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
loss_fn = loss_fn.to(device)
mae_fn = mae_fn.to(device)



epochs = 5
for epoch in range(epochs):
        for i, (X_batch, y_batch) in enumerate(tqdm.tqdm(train_loader)):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch) 
            loss = loss_fn(y_pred.view(-1), y_batch.view(-1,1))
            mae = mae_fn(y_pred.view(-1), y_batch.view(-1,1))
            rete_val = rete(y_batch[:,0,:].view(-1,1), y_pred.view(-1,1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        tqdm.tqdm.write(f"Epoch [{epoch+1}/{epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item()}, MAE: {mae.item()}, Rete: {rete_val.item()}")

RuntimeError: cuDNN version incompatibility: PyTorch was compiled  against (8, 5, 0) but found runtime version (8, 0, 1). PyTorch already comes bundled with cuDNN. One option to resolving this error is to ensure PyTorch can find the bundled cuDNN.Looks like your LD_LIBRARY_PATH contains incompatible version of cudnnPlease either remove it from the path or install cudnn (8, 5, 0)

In [ ]:
predict = []
model = model.cpu()
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predict.append(outputs[:, 0].unsqueeze(1))
predict = torch.cat(predict, dim=0)

In [ ]:
predict_list = np.concatenate([p.detach().numpy() for p in predict], axis=0)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(facecolor='white')
plt.plot(predict_list,color='darkorange')
plt.plot(y_test)
plt.xlabel('Time')
plt.ylabel("W")
plt.title("plug1-1")
plt.savefig('/home/awinlab/Documents/kevin/WGRU/pytorch/WindowGRU_plug1-1.png')
plt.show()